Before running this noetbook makes sure that the following packages are added
* `snowflake.core`
* `snowflake-ml-python`

If you get an error of that `demo_data.py`is missing then go into **Packages**->**Stage Packages** and remove it.

In [1]:
# Import python packages
import streamlit as st

# Snowflake Python API
from snowflake.core import Root
from snowflake.core.database import Database
from snowflake.core.schema import Schema
from snowflake.core.warehouse import Warehouse
from snowflake.core.stage import Stage, StageEncryption, StageDirectoryTable
from snowflake.core import Root

from snowflake.ml.feature_store import FeatureStore, CreationMode


# Get the Snowpark session
from snowflake.snowpark.context import get_active_session
session = get_active_session()


In [ ]:
db_name = "SNOWPARK_DEMO_DB" 
schema_name = "SIMPLE_ML_SCHEMA" # Data
fs_schema_name = "SIMPLE_FS_SCHEMA" # Feature Store
mr_schema_name = "SIMPLE_MR_SCHEMA" # Model Registry
wh_name = "SIMPLE_ML_WH"


In [3]:
# Create Database & Schema
root = Root(session)
ml_demo_db = Database(name=db_name)
ml_demo_db = root.databases.create(ml_demo_db, mode='if_not_exists')

ml_demo_schema = Schema(name=schema_name)
ml_demo_schema = ml_demo_db.schemas.create(ml_demo_schema, mode='or_replace')

ml_mr_schema = Schema(name=mr_schema_name)
ml_mr_schema = ml_demo_db.schemas.create(ml_mr_schema, mode='or_replace')

ml_fs_schema = Schema(name=fs_schema_name)
ml_fs_schema = ml_demo_db.schemas.create(ml_fs_schema, mode='or_replace')

code_stage = Stage(
  name="ASSETS",
  encryption=StageEncryption(type="SNOWFLAKE_SSE"), 
  directory_table=StageDirectoryTable(enable=True)
)
code_stage = ml_demo_schema.stages.create(code_stage, mode='or_replace')


# Create warehouse
ml_wh = Warehouse(
    name=wh_name, 
    warehouse_size="MEDIUM", 
    auto_suspend=600, 
    auto_resume='true', 
)

warehouses = root.warehouses
ml_wh = warehouses.create(ml_wh, mode='or_replace')

# Set context
session.use_schema(f'{db_name}.{schema_name}')
session.use_warehouse(wh_name)


# Create Feature Store
fs = FeatureStore(
    session=session, 
    database=db_name, 
    name=fs_schema_name, 
    default_warehouse=wh_name,
    creation_mode=CreationMode.CREATE_IF_NOT_EXIST,
)

In [4]:
session.use_schema(f'{db_name}.{schema_name}')
session.use_warehouse(wh_name)

session.get_fully_qualified_current_schema()


'"SNOWPARK_DEMO_DB"."SIMPLE_ML_SCHEMA"'

**Before generating data you need to upload the `demo_data.py` file to the `ASSETS` stage** and then add the file as a **Stage Package** , by gp to **Packages** -> **Stage Packages** and add `@snowpark_demo_db.simple_ml_schema.ASSETS/demo_data.py`, if you have used the default values for `db_name` and `schema_name`.

You also need to upload `plotting.py` that will be used in a Notebook later

In [ ]:
ls @snowpark_demo_db.simple_ml_schema.ASSETS/demo_data.py

In [5]:

from demo_data import generate_demo_data

session.use_schema(f'{db_name}.{schema_name}')

generate_demo_data(session, num_customers=200000, month=8, start_year=2020, end_year=2024)

Added 200000 customers to table: CUSTOMER_LIFE_TIME_VALUE
Added 200000 customers to table: CUSTOMER_GENERAL_DATA
Added 200000 customers to table: CUSTOMER_BEHAVIOR_DATA


In [8]:
# Create a Snowpark DataFrames
print('Customer Life Time Value Data:')
ltv_df = session.table(f'{db_name}.{schema_name}.CUSTOMER_LIFE_TIME_VALUE').drop('YEAR_MONTH')
ltv_df.show(n=2)

print('Customer General Data:')
cust_df = session.table(f'{db_name}.{schema_name}.CUSTOMER_GENERAL_DATA')
cust_df.show(n=2)

print('Customer Behavior Data:')
behavior_df = session.table(f'{db_name}.{schema_name}.CUSTOMER_BEHAVIOR_DATA')
behavior_df.show(n=2)

Customer Life Time Value Data:
--------------------------------------------
|"EMAIL"               |"LIFE_TIME_VALUE"  |
--------------------------------------------
|ISEglO3tBE@jRkXJ.com  |20.03              |
|blarIyWxnz@EyabS.com  |207.69             |
--------------------------------------------

Customer General Data:
------------------------------------------------------------------------------
|"EMAIL"               |"GENDER"  |"MEMBERSHIP_STATUS"  |"MEMBER_JOIN_DATE"  |
------------------------------------------------------------------------------
|ISEglO3tBE@jRkXJ.com  |MALE      |BASIC                |2020-03-21          |
|blarIyWxnz@EyabS.com  |MALE      |BRONZE               |2024-02-03          |
------------------------------------------------------------------------------

Customer Behavior Data:
-------------------------------------------------------------------------------------------------------
|"EMAIL"               |"AVG_SESSION_LENGTH_MIN"  |"AVG_TIME_ON_APP_MIN"